In [1]:
import tensorflow as tf

Building a model with the functional API works like this:

- A layer instance is callable and returns a tensor.
- Input tensors and output tensors are used to define a tf.keras.Model instance.
- This model is trained just like the Sequential model.

## 1. Load Dataset

In [3]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(60000, 28*28).astype('float32') / 255
x_test = x_test.reshape(10000, 28*28).astype('float32') / 255

## 2. Build Model with Input & Output Tensors

In [5]:
# starts from Input layer:
# sample dimension is not considered for the `shape` argument,
# but counted in the return shape: [None, 28*28] 
inputs = tf.keras.Input(shape=(28*28,)) 

# hidden layer 1
x = tf.keras.layers.Dense(256, activation='relu')(inputs)

# hidden layer 2
x = tf.keras.layers.Dense(128, activation='relu')(x)

# output layer
x = tf.keras.layers.Dense(10)(x)
outputs = tf.keras.layers.Softmax()(x)

# build model by passing inputs and outputs tansor
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [8]:
model.summary()
# tf.keras.utils.plot_model(model, 'mnist_model.png')

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
dense (Dense)                (None, 256)               200960    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
_________________________________________________________________
softmax (Softmax)            (None, 10)                0         
Total params: 235,146
Trainable params: 235,146
Non-trainable params: 0
_________________________________________________________________


## 3. Train and Test Model

In [10]:
# train
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.RMSprop(),
              metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=128, epochs=5, validation_split=0.2)

# test
test_scores = model.evaluate(x_test, y_test, verbose=2) # loss, accuracy

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 3s 64us/sample - loss: 1.4812 - accuracy: 0.9806 - val_loss: 1.4902 - val_accuracy: 0.9716
Epoch 2/5
48000/48000 [==============================] - 2s 48us/sample - loss: 1.4788 - accuracy: 0.9829 - val_loss: 1.4873 - val_accuracy: 0.9740
Epoch 3/5
48000/48000 [==============================] - 2s 49us/sample - loss: 1.4767 - accuracy: 0.9848 - val_loss: 1.4871 - val_accuracy: 0.9745
Epoch 4/5
48000/48000 [==============================] - 2s 48us/sample - loss: 1.4756 - accuracy: 0.9860 - val_loss: 1.4867 - val_accuracy: 0.9744
Epoch 5/5
48000/48000 [==============================] - 2s 49us/sample - loss: 1.4742 - accuracy: 0.9875 - val_loss: 1.4862 - val_accuracy: 0.9753
10000/10000 - 1s - loss: 1.4861 - accuracy: 0.9755
